In [1]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterstats import zonal_stats
import matplotlib.pyplot as plt
from rasterio.plot import show
import os

In [4]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\geospatial_internship\\datasets')
fire_data = pd.read_csv('telangana_fires.csv')
telangana_shape = gpd.read_file('telangana_shapefile.geojson')

os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\src\\data_preprocessing\\tsdm\\')
district_boundaries = gpd.read_file('District_Boundary.shp')

os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Classification_Fires')
fires_2021 = gpd.read_file('telanganafire.geojson')

In [5]:
fire_data['acq_date'] = pd.to_datetime(fire_data['acq_date'])
fires_2021['acq_date'] = pd.to_datetime(fires_2021['acq_date'])

In [6]:
#Create geodataframe from the data
geo_fire_data = gpd.GeoDataFrame(fire_data,geometry = gpd.points_from_xy(fire_data.longitude,fire_data.latitude), crs = {'init': 'epsg:4326'}) 

#Make sure the
geo_fire_data['geometry'] = geo_fire_data['geometry'].geometry.to_crs(epsg = 4326)
district_boundaries['geometry'] = district_boundaries['geometry'].geometry.to_crs(epsg = 4326)

c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [7]:
#Combine fires from 2021 with the rest
geo_fire_data = geo_fire_data.append(fires_2021)
geo_fire_data['fireID'] = [a for a in range(0, len(geo_fire_data))]

In [8]:
polygons = []

#-------create a buffer of square of 1km size using buffer function from shapely-----#

for i in geo_fire_data.geometry:
    p1 = i
    buffer = p1.buffer(0.004504505, cap_style = 3)         #500m = 0.004504505 and cap_style 3 is square box of same of 1km side length
    polygons.append(buffer)

#create a new column in GeoDataFrame newdf and dump polygon buffer of respective point values
geo_fire_data['geometry buffered'] = polygons 



In [9]:
geo_fire_data['acq_date'] =  pd.to_datetime(geo_fire_data['acq_date'])
geo_fire_data['year'] = (geo_fire_data['acq_date']).dt.year

In [10]:
geo_fire_data = geo_fire_data[geo_fire_data['year']<2022]
geo_fire_data

,fireID,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,geometry,geometry buffered,year
0,0,18.3777,80.5314,318.60,1.00,1.00,2015-02-01,802,Aqua,MODIS,69,6.03,302.60,7.70,D,0.0,POINT (80.53140 18.37770),"POLYGON ((80.535904505 18.382204505, 80.535904...",2015
1,1,17.5898,80.9893,315.40,1.20,1.10,2015-04-01,750,Aqua,MODIS,59,6.03,299.50,8.80,D,0.0,POINT (80.98930 17.58980),"POLYGON ((80.993804505 17.594304505, 80.993804...",2015
2,2,17.6028,81.1612,311.80,1.20,1.10,2015-05-01,527,Terra,MODIS,46,6.03,299.80,6.00,D,0.0,POINT (81.16120 17.60280),"POLYGON ((81.16570450499999 17.607304505, 81.1...",2015
3,3,17.5241,80.3135,317.00,1.10,1.00,2015-05-01,527,Terra,MODIS,69,6.03,302.00,9.30,D,0.0,POINT (80.31350 17.52410),"POLYGON ((80.318004505 17.528604505, 80.318004...",2015
4,4,17.7571,80.3404,317.20,2.50,1.50,2015-05-01,833,Aqua,MODIS,53,6.03,300.00,29.30,D,0.0,POINT (80.34040 17.75710),"POLYGON ((80.344904505 17.761604505, 80.344904...",2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,19183,NaN,NaN,316.83,1.04,1.02,2021-12-29,756,Aqua,MODIS,69,6.1NRT,299.37,8.79,D,NaN,POINT (80.83595 18.02931),"POLYGON ((80.840454505 18.033814505, 80.840454...",2021
399,19184,NaN,NaN,312.18,1.35,1.15,2021-12-29,756,Aqua,MODIS,25,6.1NRT,296.55,7.86,D,NaN,POINT (78.62585 17.26095),"POLYGON ((78.630354505 17.265454505, 78.630354...",2021
400,19185,NaN,NaN,312.74,1.15,1.07,2021-12-30,527,Terra,MODIS,64,6.1NRT,297.34,7.20,D,NaN,POINT (79.53605 18.72281),"POLYGON ((79.540554505 18.727314505, 79.540554...",2021
401,19186,NaN,NaN,312.36,1.57,1.23,2021-12-31,743,Aqua,MODIS,57,6.1NRT,299.28,9.51,D,NaN,POINT (80.53585 18.04629),"POLYGON ((80.540354505 18.050794505, 80.540354...",2021


In [11]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Landuse_Analysis')
df = pd.DataFrame(columns=geo_fire_data.columns)

for a in geo_fire_data['year'].unique():
    year = a
    year_data = geo_fire_data[geo_fire_data['year'] == year]
    if (year==2015) | (year == 2016):
        tiff = '01-01-2017.tif'
    else:
        tiff = '01-01-'+str(year)+'.tif'
    lulc = rasterio.open(tiff, mode = 'r')
    lulc_array = lulc.read(1) # landuse corresponding to each rasterpixel, so we extracted the pixel values from the raster
    # affine: 1: corresponds to the width of each pixel, 2: row rotation, 3: x-coordinate of the upper left pixel, 4: column rotation, 5: height of each pixel, 6: y-coordinate of the upper left pixel
    affine = lulc.transform
    cmap = {1: 'Water', 2: 'Trees', 4: 'Flooded Vegetation', 5: 'Crops', 7: 'Built Area', 8: 'Bare Ground', 9: 'Snow/Ice', 10: 'Clouds', 11: 'Rangeland'}
    year_data['Land use'] = 0

    for i in range(0, len(year_data)):
        test = zonal_stats(year_data['geometry'].iloc[i], lulc_array, affine = affine, geojson_out = True, nodata = lulc.nodata, categorical=True, category_map = cmap, all_touched=True)
        try: 
            year_data['Land use'].iloc[i] = list(test[0]['properties'].keys())[0]
        except:
            year_data['Land use'].iloc[i] = 'unknown'
    
    df = pd.concat([df, year_data], axis = 0)

    

c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\pandas\core\indexing.py:1965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

In [39]:
df[df['Land use']=='unknown']

,fireID,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,version,bright_t31,frp,daynight,type,geometry,geometry buffered,year,Land use,agricultural
1,1,17.5898,80.9893,315.40,1.2,1.10,2015-04-01,750,Aqua,MODIS,...,6.03,299.5,8.80,D,0.0,POINT (80.98930 17.58980),"POLYGON ((80.993804505 17.594304505, 80.993804...",2015,unknown,0
2,2,17.6028,81.1612,311.80,1.2,1.10,2015-05-01,527,Terra,MODIS,...,6.03,299.8,6.00,D,0.0,POINT (81.16120 17.60280),"POLYGON ((81.16570450499999 17.607304505, 81.1...",2015,unknown,0
198,198,17.7049,80.9274,318.10,1.1,1.00,2015-01-02,508,Terra,MODIS,...,6.03,302.4,8.60,D,0.0,POINT (80.92740 17.70490),"POLYGON ((80.93190450500001 17.709404505, 80.9...",2015,unknown,0
199,199,17.7035,80.9377,314.50,1.1,1.00,2015-01-02,508,Terra,MODIS,...,6.03,301.5,5.30,D,0.0,POINT (80.93770 17.70350),"POLYGON ((80.94220450500001 17.708004505, 80.9...",2015,unknown,0
226,226,17.6633,80.9741,322.80,1.2,1.10,2015-05-02,750,Aqua,MODIS,...,6.03,307.2,11.70,D,0.0,POINT (80.97410 17.66330),"POLYGON ((80.97860450500001 17.667804505, 80.9...",2015,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18696,18696,17.8325,81.7779,325.20,1.7,1.30,2021-05-30,737,Aqua,MODIS,...,6.03,299.4,27.70,D,0.0,POINT (81.77790 17.83250),"POLYGON ((81.782404505 17.837004505, 81.782404...",2021,unknown,0
18729,18729,17.6380,80.9699,327.10,1.5,1.20,2021-06-06,743,Aqua,MODIS,...,6.03,303.4,21.10,D,0.0,POINT (80.96990 17.63800),"POLYGON ((80.974404505 17.642504505, 80.974404...",2021,unknown,0
18730,18730,17.6404,80.9837,327.40,1.5,1.20,2021-06-06,743,Aqua,MODIS,...,6.03,303.1,21.70,D,0.0,POINT (80.98370 17.64040),"POLYGON ((80.988204505 17.644904505, 80.988204...",2021,unknown,0
18774,18774,17.6206,81.0821,321.60,1.1,1.00,2021-12-08,505,Terra,MODIS,...,6.03,299.3,9.40,D,0.0,POINT (81.08210 17.62060),"POLYGON ((81.086604505 17.625104505, 81.086604...",2021,unknown,0


In [ ]:
#ORR
#cmap = {1: 'Water', 2: 'Trees', 4: 'Flooded Vegetation', 5: 'Crops', 7: 'Built Area', 8: 'Bare Ground', 9: 'Snow/Ice', 10: 'Clouds', 11: 'Rangeland'}
#geo_fire_data['Land use 2'] = 0

#for i in range(0, len(geo_fire_data)):
#    test = zonal_stats(geo_fire_data['geometry buffered'].iloc[i], lulc_array, affine = affine, geojson_out = True, stats = 'majority', nodata = lulc.nodata, categorical=True, category_map = cmap)
#    geo_fire_data['Land use 2'].iloc[i] = test[0]['properties']['majority']

In [12]:
df['agricultural'] = 0

for i in range(0, len(df)):
    if (df['Land use'].iloc[i] == 'Crops') | (df['Land use'].iloc[i] == 'Flooded Vegetation') :
        df['agricultural'].iloc[i] = 1


In [ ]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Classification_Fires')

In [13]:
df[['fireID', 'latitude', 'longitude', 'brightness', 'scan', 'track',
       'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence',
       'version', 'bright_t31', 'frp', 'daynight', 'type', 'geometry','agricultural']].to_csv('fires_data_classified.csv')

In [16]:
df = gpd.GeoDataFrame(df,geometry = df.geometry, crs = {'init': 'epsg:4326'}) 

c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [21]:
df[['fireID', 'latitude', 'longitude', 'brightness', 'scan', 'track',
       'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence',
       'version', 'bright_t31', 'frp', 'daynight', 'type', 'geometry','agricultural']].to_file('fires_data_classified.geojson', driver="GeoJSON") 

# Classification (Using Shivangs Approach)

###Reference : https://github.com/luckyw0w/Data4Policy/tree/main/Geospatial%20Data%20Science%20Internship/ShivangPandey

So, the buffer function of geopandas (based on the shapely library) is to create a buffer polygon around a point. The first parameter of the buffer is to tell length in the degree to mark the point in a square pixel and 'cap_style' to tell the type of polygon. here 'cap_style = 3' means square polygon for buffer point. 

We need to create a polygon, because MODIS Thermal Anomalies & Fire Daily data is calculated on 1km resolution and given esri LULC data is at 10m resolution and we can say it like that 1 pixel of MODIS fire data should be as big as 10000 pixels of LULC map.
How we are going to use that? by creating a polygon as a size of 10000 pixels of LULC map.


###Reference: https://www.arcgis.com/home/item.html?id=d6642f8a4f6d4685a24ae2dc0c73d4ac

In [97]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires')
tiff = 'vmcx_mosiac_reprj.tif'

data = geo_fire_data
data['index'] = data.index

#------------------Checking probability in mosaic tiff file----------------------#
import time                                      #to calculate time taken to run the model
start_time = time.time()                         #start time of the program

index_list = []                               #declaring empty to get corresponding id of fire point
flag_list = []                                #empty list to store class of corresponding fire points based polygon
    
for j in range(len(data)):                #iterating all rows of dataframe to get point info

    stats = zonal_stats(data.iloc[j].geometry, tiff, stats="*", categorical=True)         #getting statistics from the raster point 
    i = stats[0]                                                                          #storing statsistical dictionary in a value
    if i['count'] != 0:                                                                   #check if polygon is within the tiff file or not
        index = data.iloc[j]['index']                                                     #get id of polygon
        index_list.append(index)                                                          #store id in a list
        if (5 in i) or (4 in i): 
            #print(i)                                                         #check if crop class or flooded vegitation is in polygon region or not
            flag = 1                                                                      #Mark whether occurence is in crop field
        else:
            flag = 0                                                                      # if crop class is not in polygon, return 0 pixels
        flag_list.append(flag)                                                            #store class value in a list
        
print("--- %s seconds ---" % (time.time() - start_time))                                  #print total time taken to run code 

#dictionary created wth fireID and class (1,0)
id_class = {key: value for key, value in zip(index_list, flag_list)}

#creating tuple of each key and  value pair
data_items = id_class.items()                                                
#dumping all tuples in a list
data_list = list(data_items)

#creating DataFrame with id and class values
class_df = pd.DataFrame(data_list, columns= ['fireID','class (1,0)'])


c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


--- 167.75042867660522 seconds ---


In [98]:
geo_fire_data['agricultural'] = class_df['class (1,0)']

In [99]:
geo_fire_data[['fireID', 'latitude', 'longitude', 'brightness', 'scan', 'track',
       'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence',
       'version', 'bright_t31', 'frp', 'daynight', 'type', 'geometry','agricultural']].to_csv('fires_data_classified.csv')